### The JSON outline of the research outline follows this structure:

{ 
   "title": "1. Introduction",
   "subsections": [
       {
           "title": "1.1 The Global Food Challenge and the Role of Irrigation",
           "point1": {
               "text": "...",
               "query1": "...",
               "query2": "...",
               ...
           },
           "point2": {
               "text": "...", 
               "query1": "...",
               "query2": "...",
               ...
           },
           ...
       },
       ...
   ]
}

#### The mapping from JSON to Excel is as follows:

1. The `point_text` value itself, mapped to the `text` field.
2. One or more `query` entries, mapped to the `query` field.
3. For each `query` entry, there will be the following dependent fields:
   * `doi`
   * `title` 
   * `full_text`
   * `bibtex`
   * `pdf_location`
   * `journal`
   * `citation_count`
   * `relevance_score`

To reconstruct the Excel table from the JSON, for each `point_text`, its value should be repeated for each associated `query`. This means that if there are 7 queries for a particular `point_text`, that `point_text` value should be repeated 7 times in the Excel table, with each repetition accompanied by the corresponding `query` and its dependent fields.

In [ ]:
import os
import json
import pandas as pd

# Set the path to the documents folder
documents_folder = "documents"

# Create an Excel writer object
#output_file = documents_folder + "/master.xlsx"
#writer = pd.ExcelWriter(output_file, engine="openpyxl")

# Iterate over the JSON files in the documents folder
for filename in os.listdir(documents_folder):
    if filename.startswith("outline_") and filename.endswith(".json"):
        file_path = os.path.join(documents_folder, filename)
        
        # Extract the sheet name from the filename
        sheet_name = filename.split("_")[1].split(".")[0]
        
        # Load the JSON data from the file
        with open(file_path, "r") as file:
            json_data = json.load(file)
        
        # Create an empty list to store the data for the current JSON file
        data_list = []
        
        # Extract the data from the JSON structure
        for section in json_data:
            document_title = section["title"]
            for subsection in section.get("subsections", []):
                section_title = subsection["title"]
                for point_key, point_data in subsection.items():
                    if point_key.startswith("point"):
                        for query_key, query_data in point_data.items():
                            if query_key.startswith("query"):
                                data_list.append({
                                    "document_title": document_title,
                                    "section_title": section_title,
                                    "point_text": point_data["text"],
                                    "query": query_data,
                                    "doi": "",
                                    "title": "",
                                    "full_text": "",
                                    "bibtex": "",
                                    "pdf_location": "",
                                    "journal": "",
                                    "citation_count": "",
                                    "relevance_score": ""
                                })
        
        # Create a DataFrame from the data list
        df = pd.DataFrame(data_list)
        
        # Reorder the columns
        column_order = [
            "document_title",
            "section_title",
            "point_text",
            "query",
            "doi",
            "title",
            "full_text",
            "bibtex",
            "pdf_location",
            "journal",
            "citation_count",
            "relevance_score"
        ]
        df = df[column_order]
        
        # Save the DataFrame to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Save the Excel file
writer.close()
print(f"Data from JSON files loaded and saved to {output_file}.")

In [ ]:
import openpyxl
import asyncio
import json
from llm_api_handler import LLM_APIHandler
import nest_asyncio
nest_asyncio.apply()

from prompts import get_prompt, remove_illegal_characters, outline, review_intention

class ExcelQueryProcessor:
    def __init__(self, file_path, api_key_path):
        self.file_path = file_path
        self.api_handler = LLM_APIHandler(api_key_path)

    async def process_queries(self, sheet_name, prompt_template, input_columns, output_columns):
        workbook = openpyxl.load_workbook(self.file_path)
        sheet = workbook[sheet_name]
        header_row = next(sheet.iter_rows(values_only=True))
        column_names = {cell: index for index, cell in enumerate(header_row)}
        
        query_data = []
        for row in sheet.iter_rows(values_only=True, min_row=2):
            try:
                if not row[column_names["full_text"]]:
                    continue
                query_values = {col: row[column_names[col]] for col in input_columns}
                query_data.append({"unique_identifier": row[column_names["unique_identifier"]], "query": query_values})
            except Exception as e:
                print(f"Error occurred while processing row: {str(e)}")
                break

        if not query_data:
            print("No valid queries found. Skipping API calls.")
            return

        try:
            await self.process_queries_async(query_data, prompt_template, output_columns, sheet, column_names)
            workbook.save(self.file_path)
            print("Results written to the sheet successfully.")
        except Exception as e:
            error_message = f"Error occurred during processing: {str(e)}. Aborting program."
            print(error_message)
            print("Suggested code fix:")
            print("# TODO: Implement code fix based on the error message")
            raise Exception(error_message)

    async def process_queries_async(self, query_data, prompt_template, output_columns, sheet, column_names):
        tasks = [asyncio.ensure_future(self.process_single_query(query, prompt_template, output_columns, sheet, column_names)) for query in query_data]
        await asyncio.gather(*tasks)

    async def process_single_query(self, query, prompt_template, output_columns, sheet, column_names):
        prompt = get_prompt(prompt_template, outline=outline, review_intention=review_intention, **query["query"])
        response = await self.api_handler.generate_gemini_content(prompt)
        print("Processing query:", query["query"]["unique_identifier"])
        
        try:
            await self.update_sheet(sheet, query["unique_identifier"], response, output_columns, column_names)
        except (ValueError, Exception) as e:
            error_message = f"Fatal error occurred while writing results to sheet: {str(e)}. Aborting program."
            print(error_message)
            print("Suggested code fix:")
            print("# TODO: Implement code fix based on the error message")
            raise Exception(error_message)

    async def update_sheet(self, sheet, unique_identifier, response, output_columns, column_names):
        print("Updating sheet with results...")
        for col in output_columns:
            if col not in column_names:
                sheet.cell(row=1, column=sheet.max_column + 1, value=col)
                column_names[col] = sheet.max_column

        try:
            row = next(r for r in sheet.iter_rows() if r[column_names["unique_identifier"]].value == unique_identifier)
            for col, value in response.items():
                if col in output_columns:
                    cleaned_value = remove_illegal_characters(str(value))
                    sheet.cell(row=row[0].row, column=column_names[col], value=cleaned_value)
                    print(f"Updated {col} for unique_identifier: {unique_identifier}")
        except StopIteration:
            raise ValueError(f"Row not found for unique_identifier: {unique_identifier}. Aborting write operation.")
        except Exception as e:
            raise Exception(f"Error occurred while writing to sheet: {str(e)}. Aborting write operation.")

# Usage example
file_path = "documents/master.xlsx"
api_key_path = r"C:\Users\bnsoh2\OneDrive - University of Nebraska-Lincoln\Documents\keys\api_keys.json"
sheet_name = "1"
prompt_template = "paper_analysis"
input_columns = ["point_text", "section_title", "document_title", "full_text", "unique_identifier"]
output_columns = ["relevance_score", "explanation", "extract_1", "extract_2", "Alternate_section", "limitations"]

processor = ExcelQueryProcessor(file_path, api_key_path)
try:
    asyncio.run(processor.process_queries(sheet_name, prompt_template, input_columns, output_columns))
except Exception as e:
    print(f"Fatal error: {str(e)}")

In [ ]:
import openpyxl
import asyncio
import json
from llm_api_handler import LLM_APIHandler
import nest_asyncio
nest_asyncio.apply()

from prompts import get_prompt, remove_illegal_characters, outline, review_intention

class JSONQueryProcessor:
    def __init__(self, file_path, api_key_path):
        self.file_path = file_path
        self.api_handler = LLM_APIHandler(api_key_path)

    async def process_queries(self, sheet_name, prompt_template, input_columns, output_columns):
        workbook = openpyxl.load_workbook(self.file_path)
        sheet = workbook[sheet_name]
        header_row = next(sheet.iter_rows(values_only=True))
        column_names = {cell: index for index, cell in enumerate(header_row)}

        query_data = []
        for row in sheet.iter_rows(values_only=True, min_row=2):
            try:
                if not row[column_names["full_text"]]:
                    continue
                query_values = {col: row[column_names[col]] for col in input_columns}
                query_data.append(query_values)
            except Exception as e:
                print(f"Error occurred while processing row: {str(e)}")
                break

        if not query_data:
            print("No valid queries found. Skipping API calls.")
            return

        try:
            result = await self.process_queries_async(query_data, prompt_template, output_columns)
            self.write_to_json(result)
            print("Results written to the JSON file successfully.")
        except Exception as e:
            error_message = f"Error occurred during processing: {str(e)}. Aborting program."
            print(error_message)
            print("Suggested code fix:")
            print("# TODO: Implement code fix based on the error message")
            raise Exception(error_message)

    async def process_queries_async(self, query_data, prompt_template, output_columns):
        tasks = [asyncio.ensure_future(self.process_single_query(query, prompt_template, output_columns)) for query in query_data]
        results = await asyncio.gather(*tasks)
        return self.structure_results(query_data, results, output_columns)

    async def process_single_query(self, query, prompt_template, output_columns):
        prompt = get_prompt(prompt_template, outline=outline, review_intention=review_intention, **query)
        response = await self.api_handler.generate_gemini_content(prompt)
        print("Processing query:", query["unique_identifier"])
        return response

    def structure_results(self, query_data, results, output_columns):
        structured_data = {}
        for query, result in zip(query_data, results):
            document_title = query["document_title"]
            section_title = query["section_title"]
            point_text = query["point_text"]

            if document_title not in structured_data:
                structured_data[document_title] = {}
            if section_title not in structured_data[document_title]:
                structured_data[document_title][section_title] = {}
            if point_text not in structured_data[document_title][section_title]:
                structured_data[document_title][section_title][point_text] = {}

            for col in output_columns:
                if col in result:
                    cleaned_value = remove_illegal_characters(str(result[col]))
                    structured_data[document_title][section_title][point_text][col] = cleaned_value

        return structured_data

    def write_to_json(self, data):
        print("Writing results to JSON file...")
        try:
            with open("output.json", "w") as json_file:
                json.dump(data, json_file, indent=4)
        except IOError as e:
            print(f"Error writing to JSON file: {str(e)}")
            print("Attempting to write results in smaller chunks...")
            self.write_to_json_in_chunks(data)

    def write_to_json_in_chunks(self, data, chunk_size=100):
        with open("output.json", "w") as json_file:
            json_file.write("{")
            for i, (document_title, document_data) in enumerate(data.items()):
                if i > 0:
                    json_file.write(",")
                json.dump(f'"{document_title}"', json_file)
                json_file.write(":")
                json.dump(document_data, json_file, indent=4)
            json_file.write("}")

# Usage example
file_path = "documents/master.xlsx"
api_key_path = r"C:\Users\bnsoh2\OneDrive - University of Nebraska-Lincoln\Documents\keys\api_keys.json"
sheet_name = "1"
prompt_template = "paper_analysis"
input_columns = ["point_text", "section_title", "document_title", "full_text", "unique_identifier"]
output_columns = ["relevance_score", "explanation", "extract_1", "extract_2", "Alternate_section", "limitations"]

processor = JSONQueryProcessor(file_path, api_key_path)
try:
    asyncio.run(processor.process_queries(sheet_name, prompt_template, input_columns, output_columns))
except Exception as e:
    print(f"Fatal error: {str(e)}")

c:\Users\bnsoh2\Desktop\venvs\automated_lit_reviews\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:root:Error in Gemini API call: 429 Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:780208693127'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/generate_content_requests"
}
metadata {
  key: "quota_location"
  value: "us-east2"
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadat

Error occurred during processing: 429 Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:780208693127'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/generate_content_requests"
}
metadata {
  key: "quota_location"
  value: "us-east2"
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadata {
  key: "consumer"
  value: "projects/780208693127"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
]. Aborting program.
Suggested code fix:
# TODO: Implement code fix based on the error messag

ERROR:root:Error in Gemini API call: 429 Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:780208693127'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/generate_content_requests"
}
metadata {
  key: "quota_location"
  value: "us-east2"
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadata {
  key: "consumer"
  value: "projects/780208693127"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
]
ERROR:root:Error in Gemini API call: 429 Quota exceeded for quota metric 'Generate Conte

{
"explanation": "This paper does not directly address the challenges of feeding a growing population with finite resources, which is mentioned in the literature review outline. The focus of the paper is on waste management and pollution control, rather than irrigation management for agricultural purposes.",
"extract_1": null,
"extract_2": null,
"relevance_score": 0.0,
"Alternate_section": null,
"limitations": "The paper does not provide any insights into the role of irrigation in enhancing crop yields and agricultural productivity to meet the growing demand for food.",
"inline_citation": null,
"apa_citation": null
}
Processing query: 3
{
"explanation": "Howell focuses in this paper on providing a general state of the art review of irrigation scheduling and how it has developed since the early adoption of computers to it.  This paper includes his view on the issues and problems of irrigation scheduling as it existed at the time, but it does not specifically address how it might relate 

ERROR:root:Error in Gemini API call: 429 Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:780208693127'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/generate_content_requests"
}
metadata {
  key: "quota_location"
  value: "us-east2"
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadata {
  key: "consumer"
  value: "projects/780208693127"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
]


...

{
"explanation": "This paper focused on assessing the temporal change characteristics of national water footprint (NWF) for single-rice (SR), early-rice (ER) and late-rice (LR) in different agro-ecological zones across China during 2001–2010 for the first time. The results exhibited that NWF of rice was an estimated 304,848 million cubic meters (MCM) per year. The SR accounted for the greatest portion of NWF, followed by ER and LR. The NWF rank was SR-V N SR-I N ER-VI N SR-IV N LR-III N LR-VI N SR-II N ER-III. The blue water footprint (WFb) presents decreasing trends in most agro-ecological zones (SR-I, SR-II, SR-IV, ER-III and LR-VI), while green water footprint (WFg) exhibits increasing trends within these regions.",
"extract_1": "The NWF of rice was an estimated 304,848 million cubic meters (MCM) per year. The SR accounted for the greatest portion of NWF, followed by ER and LR. The NWF rank was SR-V N SR-I N ER-VI N SR-IV N LR-III N LR-VI N SR-II N ER-III.",
"extract_2": "The blue w

ERROR:root:Error in Gemini API call: Invalid JSON format in the response


{
"explanation": "The paper investigates the influence of magnetically treated water on the growth, nutritional status, and yield of lettuce plants under different irrigation rates. The study aims to determine whether magnetic water treatment can improve water utilization efficiency in lettuce production. The results suggest that magnetically treated water enhances the concentrations of nitrogen and phosphorus in leaves, leading to higher nitrogen assimilation and increased agronomical characteristics (leaf number, fresh and dry shoot weight, and fresh and dry root weight). The use of magnetically treated water was found to have beneficial effects on lettuce plants, improving their nutritional status and yield while also potentially increasing water use efficiency and optimizing irrigation management.",
"extract_1": "The conclusions of this study showcase that magnetically treated water has beneficial effects on lettuce plants, improving their nutritional status and yield.",
"extract_2

ERROR:root:Error in Gemini API call: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting


<response_format>
{
"explanation": "The paper analyzes irrigation water withdrawal and use characteristics and discusses regional irrigator behavior patterns of commercial agriculture fields for three consecutive years in the Republican River Basin (RRB), Nebraska, United States. Irrigation water requirements were estimated, and the datasets were compared and assessed against irrigation water depths directly measured at producer fields. The results showed that actual irrigation applications by producers did not change considerably despite reductions in irrigation water requirement over the study period, and in 80% of the cases, irrigation applications exceeded irrigation water requirements. Additionally, some fields exceeded the mean annual irrigation depth allowed by the moratorium imposed by the Natural Resources Districts (NRDs) in the area. These findings indicate that irrigation decisions are still largely driven by non-scientific and/or technical methods.",
"extract_1": "\"Despit